# Wolfers (2006) Replication Benchmark {#sec-wolfers}

This chapter benchmarks DID estimators across **Stata**, **R**, and **Python** using the Wolfers (2006) divorce rate dataset from the DiD textbook.

## Overview

We replicate the analysis from the textbook `solution.do` file using four modern DID estimators:

| Estimator | Reference | Stata | R | Python |
|-----------|-----------|:-----:|:-:|:------:|
| De Chaisemartin & D'Haultfoeuille | @dechaisemartin2024 | `did_multiplegt_dyn` | `DIDmultiplegtDYN` | `did-multiplegt-dyn` |
| Callaway & Sant'Anna | @callaway2021 | `csdid` | `did` | `csdid` |
| Borusyak, Jaravel & Spiess | @borusyak2024 | `did_imputation` | `didimputation` | **N/A** |
| Sun & Abraham | @sun2021 | `eventstudyinteract` | `fixest::sunab` | `pyfixest` |

## Dataset

**Wolfers (2006)**: Panel data on U.S. state divorce rates, 1956-1988.

- **51 states** (including DC)
- **33 years** of data
- **1,683 observations**
- Treatment: Unilateral divorce law adoption (staggered timing)


In [ ]:
#| label: load-data
#| echo: true
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load combined results from all platforms
results = pd.read_csv('CX/runtime_all_platforms.csv')

print(f"Total benchmark runs: {len(results)}")
print(f"\nPlatforms: {', '.join(results['platform'].unique())}")
print(f"Estimators: {', '.join(results['package_std'].unique())}")

## Benchmark Specifications

All estimators use the same core specification:

- **Outcome**: `div_rate` (divorce rate)
- **Group**: `state`
- **Time**: `year`
- **Treatment**: `udl` (unilateral divorce law indicator)
- **Effects**: 13 dynamic effects (post-treatment)
- **Placebos**: 13 pre-treatment periods
- **Weights**: `stpop` (state population)

### Data Scaling

We test three dataset sizes to evaluate scalability:

| Scale | Observations | States | Description |
|-------|-------------|--------|-------------|
| Original | 1,683 | 51 | Original Wolfers data |
| 100x | 168,300 | 5,100 | Synthetic replication |
| 1000x | 1,683,000 | 51,000 | Large-scale test |

## Results: Runtime Comparison


In [ ]:
#| label: fig-runtime-comparison
#| fig-cap: Runtime comparison across platforms and dataset sizes

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

scenarios = ['Original (1.7K)', '100x (168K)', '1000x (1.68M)']
completed = results[results['status'] == 'completed'].copy()

for idx, scenario in enumerate(scenarios):
    ax = axes[idx]
    scenario_data = completed[completed['scenario'] == scenario]

    if len(scenario_data) > 0:
        pivot = scenario_data.pivot_table(
            values='time_seconds',
            index='package_std',
            columns='platform',
            aggfunc='first'
        )
        # Reorder columns
        col_order = ['Stata', 'R', 'Python']
        pivot = pivot[[c for c in col_order if c in pivot.columns]]
        pivot.plot(kind='bar', ax=ax, width=0.8, color=['#1f77b4', '#2ca02c', '#ff7f0e'])
        ax.set_title(f'{scenario}', fontsize=12, fontweight='bold')
        ax.set_xlabel('')
        ax.set_ylabel('Runtime (seconds)' if idx == 0 else '')
        ax.legend(title='Platform' if idx == 2 else '', loc='upper left')
        ax.tick_params(axis='x', rotation=45)

plt.suptitle('DID Estimator Runtime: Stata vs R vs Python', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Runtime Tables

### By Platform


In [ ]:
#| label: tbl-runtime-platform
#| tbl-cap: Runtime (seconds) by platform and estimator

for platform in ['Stata', 'R', 'Python']:
    platform_data = completed[completed['platform'] == platform]
    if len(platform_data) > 0:
        print(f"\n### {platform}\n")
        pivot = platform_data.pivot_table(
            values='time_seconds',
            index='package_std',
            columns='scenario',
            aggfunc='first'
        )
        col_order = ['Original (1.7K)', '100x (168K)', '1000x (1.68M)']
        pivot = pivot[[c for c in col_order if c in pivot.columns]]
        print(pivot.round(2).to_markdown())

### Cross-Platform Comparison by Estimator


In [ ]:
#| label: tbl-cross-platform
#| tbl-cap: Cross-platform runtime comparison for did_multiplegt_dyn

print("\n### did_multiplegt_dyn (De Chaisemartin & D'Haultfoeuille)\n")
dmdyn_data = completed[completed['package_std'] == 'did_multiplegt_dyn']
pivot = dmdyn_data.pivot_table(
    values='time_seconds',
    index='platform',
    columns='scenario',
    aggfunc='first'
)
col_order = ['Original (1.7K)', '100x (168K)', '1000x (1.68M)']
pivot = pivot[[c for c in col_order if c in pivot.columns]]
pivot = pivot.reindex(['Stata', 'R', 'Python'])
print(pivot.round(2).to_markdown())

## Key Findings

### Scaling Performance at 1.68M Observations


In [ ]:
#| label: fig-large-scale
#| fig-cap: Runtime at 1.68M observations

large_data = completed[completed['scenario'] == '1000x (1.68M)'].copy()

# Exclude R's did_imputation outlier for visualization
large_data_viz = large_data[~((large_data['platform'] == 'R') &
                               (large_data['package_std'] == 'did_imputation (BJS)'))]

fig, ax = plt.subplots(figsize=(10, 5))
pivot = large_data_viz.pivot_table(
    values='time_seconds',
    index='package_std',
    columns='platform',
    aggfunc='first'
)
col_order = ['Stata', 'R', 'Python']
pivot = pivot[[c for c in col_order if c in pivot.columns]]
pivot.plot(kind='bar', ax=ax, width=0.8, color=['#1f77b4', '#2ca02c', '#ff7f0e'])
ax.set_xlabel('Estimator')
ax.set_ylabel('Runtime (seconds)')
ax.set_title('Runtime at 1.68M Observations\n(excluding R didimputation outlier: 44,680s)')
ax.legend(title='Platform')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

### Performance Summary


In [ ]:
#| label: summary-stats
#| echo: false

# Best performers at 1.68M
large = completed[completed['scenario'] == '1000x (1.68M)']
print("**Fastest at 1.68M observations:**\n")
for est in large['package_std'].unique():
    est_data = large[large['package_std'] == est].sort_values('time_seconds')
    if len(est_data) > 0:
        fastest = est_data.iloc[0]
        print(f"- **{est}**: {fastest['platform']} ({fastest['time_seconds']:.1f}s)")

## Package Availability

| Estimator | Stata | R | Python |
|-----------|:-----:|:-:|:------:|
| did_multiplegt_dyn | `did_multiplegt_dyn` | `DIDmultiplegtDYN` | `did-multiplegt-dyn` |
| Callaway-Sant'Anna | `csdid` (bootstrap) | `did` | `csdid` |
| Borusyak et al. | `did_imputation` | `didimputation` | **Not available** |
| Sun-Abraham | `eventstudyinteract` | `fixest::sunab` | `pyfixest` |

### Notes

1. **Stata's `csdid`** uses bootstrap inference by default, making it slower than analytical SE methods
2. **R's `didimputation`** has extremely poor scaling (44,680s at 1.68M rows) - likely a bug or memory issue
3. **Python lacks `did_imputation`** - no implementation of Borusyak et al. (2024) exists
4. **`did_multiplegt_dyn`** shows consistent performance across all three platforms

## Reproducibility

The benchmark scripts are available in the `CX/` directory:

- `CX/benchmark_wolfers_python.ipynb` - Python benchmark
- `CX/benchmark_wolfers_complete.R` - R benchmark
- `CX/benchmark_wolfers_stata.do` - Stata benchmark

All scripts use a 5-minute (300 second) timeout per estimator.

## References

::: {#refs}
:::